In [1]:
from IPython.core.display import display,HTML

display(HTML("<style>.container { width: 90% !important;}</style>"))

/var/folders/s2/bjtyvybn0kv3rwjkn5xht9_m0000gn/T/ipykernel_4723/2814205877.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display,HTML


In [2]:
import pandas as pd
import govt_func as gf
import datetime
from imp import reload

import numpy as np
from dateutil.relativedelta import *

# Import Data

In [4]:
deferred_uri = "data/deferred_income.xlsx"
df_deferred = pd.read_excel(deferred_uri)
profit_uri = "data/profit.xlsx"
df_profit = pd.read_excel(profit_uri)
df_deferred = df_deferred.drop("证券名称", axis=1)
df_profit = df_profit.drop("证券名称", axis=1)
df_deferred = df_deferred.rename(columns={"CITICS_INDUSTRY": "INDUSTRY"})
df_profit = df_profit.rename(columns={"CITICS_INDUSTRY": "INDUSTRY"})
df_deferred = df_deferred.dropna(axis=0, subset="INDUSTRY")
df_profit = df_profit.dropna(axis=0, subset="INDUSTRY")
df_deferred = df_deferred.drop(
    df_deferred.iloc[:, 2:].filter(like="Y", axis=1).columns, axis=1
)  # exclude annual statement
df_profit = df_profit.drop(
    df_profit.iloc[:, 2:].filter(like="Y", axis=1).columns, axis=1
)  # exclude annual statement
df_profit

,SECU_CODE,INDUSTRY,2011Q1,2011Q2,2011Q3,2012Q1,2012Q2,2012Q3,2013Q1,2013Q2,...,2019Q3,2020Q1,2020Q2,2020Q3,2021Q1,2021Q2,2021Q3,2022Q1,2022Q2,2022Q3
0,000001.SZ,银行,3.069099e+09,6.065846e+09,9.924443e+09,4.477351e+09,8.878649e+09,1.341211e+10,4.718000e+09,9.906000e+09,...,3.067600e+10,1.095800e+10,1.758700e+10,2.882600e+10,1.287400e+10,2.234100e+10,3.678700e+10,1.618400e+10,2.778300e+10,4.611200e+10
1,000002.SZ,房地产,1.674455e+09,4.451809e+09,5.650196e+09,2.058143e+09,6.048672e+09,8.165983e+09,2.394886e+09,7.133412e+09,...,4.182479e+10,4.795679e+09,2.645044e+10,4.183800e+10,4.108071e+09,2.208087e+10,3.310130e+10,4.081142e+09,2.435920e+10,3.773112e+10
2,000004.SZ,计算机,2.080828e+06,4.696894e+06,6.829496e+06,1.558349e+06,2.517899e+06,5.536555e+06,3.720557e+06,5.989674e+06,...,-5.846578e+06,4.403971e+06,3.489948e+06,6.051302e+07,4.846545e+06,6.768830e+06,3.444398e+07,-1.309889e+07,-4.101779e+07,-6.634204e+07
3,000005.SZ,电力及公用事业,-6.291033e+06,2.712336e+07,1.365665e+07,-1.222558e+07,-2.925819e+07,-1.338760e+07,-9.235390e+06,-2.245686e+07,...,7.198863e+07,-2.230184e+07,-3.336571e+07,-2.759660e+07,8.595700e+06,2.005893e+08,2.032864e+08,8.476993e+05,3.500718e+06,7.114178e+06
4,000006.SZ,房地产,7.129346e+07,1.888583e+08,3.602888e+08,1.187848e+08,3.855712e+08,4.844263e+08,8.335351e+07,3.547500e+08,...,5.909587e+08,1.829440e+08,3.056448e+08,6.234316e+08,3.655433e+08,5.056996e+08,7.354123e+08,5.677890e+07,3.164525e+08,2.806002e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5005,688799.SH,医药,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8.761440e+07,2.915974e+07,6.388089e+07,1.025185e+08,3.881189e+07,7.983247e+07,1.203837e+08,3.984571e+07,9.557748e+07,1.375809e+08
5006,688800.SH,电子,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,9.999367e+06,4.943665e+07,7.012116e+07,1.874455e+07,4.223445e+07,7.987381e+07,6.377653e+07,1.431755e+08,2.241252e+08
5007,688819.SH,电力设备及新能源,NaN,2.930077e+08,5.574871e+08,2.226396e+08,4.854227e+08,7.762266e+08,1.371183e+08,1.226802e+08,...,1.132626e+09,4.031860e+08,9.755104e+08,1.845066e+09,5.226400e+08,8.412500e+08,1.307616e+09,4.854114e+08,8.622961e+08,1.677084e+09
5008,688981.SH,电子,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.867820e+08,4.486887e+08,1.424889e+09,3.314351e+09,9.197230e+08,5.074229e+09,7.555459e+09,3.650392e+09,7.866118e+09,1.177550e+10


# Get DEFERRED in Quarter 

In [5]:
df = df_deferred.set_index(["SECU_CODE","INDUSTRY"])

df = df.stack()
df = df.reset_index()

df_deferred = df.rename(columns = {"level_2":"QUARTER", 0: "DEFERRED"})
df_deferred

,SECU_CODE,INDUSTRY,QUARTER,DEFERRED
0,000004.SZ,计算机,2020Q1,624527.72
1,000004.SZ,计算机,2020Q2,624527.72
2,000004.SZ,计算机,2020Q3,624527.72
3,000004.SZ,计算机,2021Q1,471532.49
4,000004.SZ,计算机,2021Q2,471532.49
...,...,...,...,...
75447,689009.SH,机械,2021Q2,45201982.08
75448,689009.SH,机械,2021Q3,44515389.58
75449,689009.SH,机械,2022Q1,46142204.58
75450,689009.SH,机械,2022Q2,45270629.01
